In [1]:
pwd

'/Users/aidasaglinskas/Desktop/BC-MRI-AE/Colab Notebooks'

In [2]:
import numpy as np
import pickle
import pandas as pd
from matplotlib import  pyplot as plt
import  seaborn as sns
from helper_funcs import *

from tqdm import tqdm_notebook as tqdm
from datetime import datetime

In [3]:
df = pd.read_csv('../Data/ABIDE_legend_S982_pca.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,index,BIDS_ID,Anonymized ID,Subject Type,Visit,Days since enrollment,SubID,DxGroup,...,Medication Name.1,Off stimulants at EPI scan?,ScanSite,ScannerType,ScannerID,ScanSiteID,ADOS_PCA,ADI_PCA,Vineland_PCA,WISC_PCA
0,0,0,0,sub-001,A00033348,PATIENT,session_1,8,51320,1,...,NaN,NaN,mpg,verio,7,3,0,0,0,0
1,1,1,1,sub-002,A00033398,CONTROL,session_1,8,51373,2,...,NaN,NaN,mpg,verio,7,3,0,0,0,0
2,2,2,2,sub-003,A00033352,PATIENT,session_1,8,51324,1,...,NaN,NaN,mpg,verio,7,3,0,0,0,0
3,3,3,3,sub-004,A00033389,CONTROL,session_1,8,51364,2,...,NaN,NaN,mpg,verio,7,3,0,0,0,0
4,4,4,4,sub-005,A00033354,PATIENT,session_1,8,51326,1,...,NaN,NaN,mpg,verio,7,3,0,0,0,0


In [4]:
subs = np.load('../Data/ABIDE-subs-982.npz')['subs']

In [5]:
assert len(subs)==len(df), 'diff lenghts'
assert all([subs[s]==df['BIDS_ID'].values[s] for s in range(len(df))]), 'bad order'

In [6]:
patients = df['DxGroup'].values==1

In [10]:
# fn = '../Data/embedded_data.pickle'
# data = pickle.load(open(fn,'rb'))
#np.array(list(data.keys()))

In [12]:
# for key in data.keys():
#     try:
#         print(f'{key} | {data[key].shape}')
#     except:
#         pass

In [13]:
#data_latent = np.load('../Data/latent_vecs100.npz')
data_latent = np.load('../Data/latent_vecs2.npz')
list(data_latent.keys())
print(data_latent['salient_vec_abide'].shape)

(10, 982, 16)


In [14]:
data_latent['salient_vec_abide'][0,:,:]

array([[-0.4272499 ,  0.2735272 , -2.7508783 , ..., -0.91731685,
         0.1433426 , -1.1449697 ],
       [-1.4786693 , -0.7763392 , -1.2325766 , ...,  0.0499858 ,
         0.5017805 ,  0.6354925 ],
       [-2.1715796 ,  1.5461673 ,  0.53020257, ..., -0.15939587,
        -0.6093791 , -0.77931774],
       ...,
       [-0.16703203,  0.08689937, -0.4655738 , ...,  1.1736007 ,
        -0.59409565, -1.0329659 ],
       [-1.2406087 , -0.45388728, -0.7552429 , ..., -1.2301507 ,
         0.5705857 , -2.9799838 ],
       [-1.0072553 , -0.18212083, -0.8699565 , ...,  1.8765252 ,
        -0.291705  , -0.71864563]], dtype=float32)

In [15]:
now = datetime.now
t0 = now()
print(t0)

2021-08-26 15:00:22.391390


In [16]:
%%capture
n = 10 
umap_mat = np.zeros((n,470,2))
for i in tqdm(range(n)):
    umap_mat[i,:,:] = data['reducer_sl_abide'].transform(data_latent['salient_vec_abide'][i,patients,:])

NameError: name 'data' is not defined

In [17]:
print(now())
print(now()-t0)

2021-08-26 15:00:24.702670
0:00:02.311366


In [ ]:
umap_mat.shape

In [ ]:
y = [get_triu(np.corrcoef(umap_mat[:,:,0])).mean(),get_triu(np.corrcoef(umap_mat[:,:,1])).mean()]
se = [get_triu(np.corrcoef(umap_mat[:,:,0])).std(),get_triu(np.corrcoef(umap_mat[:,:,1])).std()]
plt.bar([0,1],y)
plt.errorbar([0,1],y,se,fmt='r.')

In [ ]:
def plot_error_bar(y,xlbls=None,title=None):
    #y = [r_x,r_y]
    xs = np.arange(len(y))
    m = [val.mean() for val in y]
    se = [val.std() for val in y]

    plt.bar(xs,m)
    plt.errorbar(xs,m,se,fmt='r.')
    if xlbls:
        plt.xticks(xs,labels=xlbls)
    if title:
        plt.title(title,fontsize=12)

In [ ]:
def plot_scatters(x,ys):
    plt.figure(figsize=np.array((10,5))*.85)
    plt.subplot(1,2,1)
    #x = vec[v]
    #y = umap_mat[i,v,0]
    #print(x.shape)
    #print(ys[0].shape)
    plt.scatter(x,ys[0])

    m,b = np.polyfit(x, ys[0], 1)
    plt.plot(x,m*x+b,'r-',linewidth=3)

    plt.subplot(1,2,2)
    #x,y = vec[v],umap_mat[i,v,1]
    plt.scatter(x,ys[1])

    m,b = np.polyfit(x, ys[1], 1)
    plt.plot(x,m*x+b,'r-',linewidth=3)

In [ ]:
stats.ttest_rel(abs(r_x),abs(r_y))

In [ ]:
key = 'FIQ'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
import scipy.stats as stats

In [ ]:
key = 'Sex'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
key = 'AgeAtScan'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
key = 'VINELAND_Receptive_Vscore'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
key = 'SRS_awareness'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
key = 'SRS_RawTotal'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
key = 'ADOS_Social'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])

In [ ]:
# plt.scatter(vec[v],umap_mat[i,v,1])
# m, b = np.polyfit(vec[v],umap_mat[i,v,1],1)
# plt.plot(vec[v], m*vec[v] + b,'r-')

In [ ]:
key = 'ADOS_Total'
vec = df[key].values[patients]
v = ~np.isnan(vec)
r_x = np.array([np.corrcoef(vec[v],umap_mat[i,v,0])[0,1] for i in range(n)])
r_y = np.array([np.corrcoef(vec[v],umap_mat[i,v,1])[0,1] for i in range(n)])
plot_error_bar([r_x,r_y],xlbls=['X','Y'],title=key)
plot_scatters(vec[v],[umap_mat[i,v,0],umap_mat[i,v,1]])